In [1]:
import psycopg2
import pandas as pd

In [2]:
hostname = "pgsql-siackathon.postgres.database.azure.com"
database = "postgres"
port = 5432
user = "siackathon"
password = "gFvwZjgL06u5uPyh7PnG"

connection = psycopg2.connect(
    host=hostname,
    database=database,
    user=user,
    password=password,
    port=port,
    sslmode='require'  # Ensures SSL connection
)
print("Connection to the database established successfully!")

Connection to the database established successfully!


In [3]:
query = "SELECT * FROM flotation_data;"
data = pd.read_sql_query(query, connection)
print("Data retrieved successfully!")

/tmp/ipykernel_2665/3321281568.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query(query, connection)


Data retrieved successfully!


In [4]:
df = data.copy()

In [5]:
# transform timestamp and sort
def transform_and_sort(df: pd.DataFrame, date_var) -> pd.DataFrame:
    df[date_var] = pd.to_datetime(df[date_var])
    df = df.sort_values(by=date_var)
    df = df.reset_index(drop=True)
    return df
df = transform_and_sort(df, 'ts')

In [6]:
df.ts.dt.year.unique()

array([1824, 2024], dtype=int32)

In [7]:
df.shape

(717453, 30)

In [8]:
df = df[df.ts>'2000']
df.shape

(708812, 30)

#### EDA

In [9]:
def find_numerical_columns_with_str_values(data: pd.DataFrame):
    """
    Find list of numerical columns in the DataFrame that contain string values.
    Args:
        data (pd.DataFrame): The input DataFrame.

    Returns:
        List[str]: A list of names of numerical columns that contain string values.
    """
    categorical_cols = data.select_dtypes(include=['object']).columns
    cols_with_str_values = []
    for col in categorical_cols:
        # Convert to numeric and check for NaNs
        is_non_numeric = pd.to_numeric(data[col], errors='coerce').isna().sum()
        if is_non_numeric != 0:
            cols_with_str_values.append(col)    
            
    return cols_with_str_values

def find_str_values_in_num_cols(data: pd.DataFrame) -> pd.DataFrame:

    # string_cols = find_numerical_columns_with_str_values(data)
    string_cols = ['% Iron Feed']
    print('We have ', len(string_cols), 'numeric variables with string data ')
    string_values_dict = {}
    for col in string_cols:
        # Filter the column for string values
        string_values = data[col].apply(lambda x: isinstance(x, str))

        # Extract unique string values and their counts
        unique_strings = data[string_values][col].value_counts()
        if not unique_strings.empty:
            string_values_dict[col] = unique_strings.to_dict()

    # Transform the dictionary into a DataFrame
    table = []
    for col, values in string_values_dict.items():
        for string_value, count in values.items():
            percentage = count/len(data) * 100
            table.append([col, string_value, count, percentage])

    string_values_df = pd.DataFrame(table, columns=['Column', 'String Value', 'Count', 'Percentage'])
    return string_values_df  

In [10]:
string_values_df = find_str_values_in_num_cols(df)
string_values_df

We have  1 numeric variables with string data 


,Column,String Value,Count,Percentage
0,% Iron Feed,64.03,138207,19.498400
1,% Iron Feed,57.46,44472,6.274160
2,% Iron Feed,48.81,33523,4.729463
3,% Iron Feed,52.61,14099,1.989103
4,% Iron Feed,ERROR,13535,1.909533
...,...,...,...,...
264,% Iron Feed,60.66,1036,0.146160
265,% Iron Feed,55.09,1033,0.145737
266,% Iron Feed,51.27,1033,0.145737
267,% Iron Feed,55.71,1027,0.144890


In [11]:
df[df['% Iron Feed'] == 'ERROR']

,ts,% Iron Feed,% Silica Feed,Starch Flow,Amina Flow,Ore Pulp Flow,Ore Pulp pH,Ore Pulp Density,Cell Spin Factor,Bubble Size,...,Flotation Column 02 Level,Flotation Column 03 Level,Flotation Column 04 Level,Flotation Column 05 Level,Flotation Column 06 Level,Flotation Column 07 Level,% Iron Concentrate,% Silica Concentrate,% Iron Concentrate 2,% Silica Concentrate 2
8646,2024-03-15 01:01:40,ERROR,ERROR,3079.10,564.70,396.53,10.07,1.74,1.04,0.9,...,AIR: 443.27,AIR: 460.45,AIR: 439.92,AIR: 451.59,433.54,425.46,66.91,1.31,66.94,NaN
8656,2024-03-15 01:05:00,ERROR,ERROR,3562.70,560.36,394.69,10.08,1.74,1.05,0.9,...,AIR: 416.42,AIR: 435.96,AIR: 459.92,AIR: 484.36,457.85,473.85,66.91,1.31,66.94,NaN
8678,2024-03-15 01:12:20,ERROR,ERROR,3725.78,570.13,403.89,10.11,1.74,0.96,0.8,...,AIR: 495.44,AIR: 458.67,AIR: 428.4,AIR: 385.22,445.75,432.02,66.91,1.31,66.94,NaN
8704,2024-03-15 01:21:20,ERROR,ERROR,3055.86,596.71,398.44,10.13,1.75,0.91,0.7,...,AIR: 264.88,AIR: 464.68,AIR: 476.56,AIR: 469.95,460.68,418.79,66.91,1.31,66.94,NaN
8708,2024-03-15 01:22:40,ERROR,ERROR,3091.11,588.72,398.38,10.13,1.75,0.92,0.8,...,AIR: 514.75,AIR: 445.72,AIR: 418.71,AIR: 449.06,497.86,517.93,66.91,1.31,66.94,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717168,2024-09-10 07:18:00,ERROR,ERROR,1145.56,254.67,384.77,9.14,1.52,1.01,0.9,...,AIR: 645.99,AIR: 561.25,AIR: 422.72,AIR: 369.45,432.41,301.86,63.00,4.40,57.13,5.15
717219,2024-09-10 07:35:00,ERROR,ERROR,2043.88,287.81,384.51,9.25,1.52,0.95,0.9,...,AIR: 612.28,AIR: 666.35,AIR: 398.7,AIR: 437.87,540.96,376.81,63.00,4.39,57.13,5.15
717238,2024-09-10 07:41:20,ERROR,ERROR,805.56,290.83,384.42,9.22,1.52,0.94,0.8,...,AIR: 594.58,AIR: 596.89,AIR: 423.04,AIR: 507.34,494.83,477.53,63.00,4.38,57.13,5.15
717279,2024-09-10 07:55:00,ERROR,ERROR,2017.48,297.35,384.21,9.14,1.52,0.89,0.8,...,AIR: 589.31,AIR: 532.57,AIR: 527.97,AIR: 506.95,683.88,636.62,63.00,4.37,57.13,5.15


#### Preprocessing

In [12]:
cat_variables = df.select_dtypes(include=['object']).columns
# df_filtered = df[df['% Iron Feed'] != 'ERROR']
df_filtered = df
for col in cat_variables[2:]:
    df_filtered[col] = df_filtered[col].apply(lambda x: x[5:])
    
df_filtered[cat_variables] = df_filtered[cat_variables].apply(pd.to_numeric, errors='coerce')  

In [13]:
# remove rows with nan in target variable
df_filtered = df_filtered.dropna(subset=['% Silica Concentrate 2'])

In [14]:
from sklearn.experimental import enable_iterative_imputer  # Enable iterative imputer
from sklearn.impute import IterativeImputer
# Define the columns you want to exclude from imputation
exclude_columns = ['ts', '% Iron Concentrate', '% Silica Concentrate', '% Iron Concentrate 2',
       '% Silica Concentrate 2']
# Separate the columns to impute (exclude the specified columns)
columns_to_impute = [col for col in df_filtered.columns if col not in exclude_columns]
# Initialize the IterativeImputer (MICE imputation)
imputer = IterativeImputer(max_iter=10, random_state=42)
 
# Apply imputation on the selected columns (those with missing values)
df_filtered[columns_to_impute] = imputer.fit_transform(df_filtered[columns_to_impute])
 
# Check if there are any missing values left
print(df_filtered.isnull().sum())  # Check if there are still missing values
print(df_filtered.head())  # Preview the imputed dataset

ts                              0
% Iron Feed                     0
% Silica Feed                   0
Starch Flow                     0
Amina Flow                      0
Ore Pulp Flow                   0
Ore Pulp pH                     0
Ore Pulp Density                0
Cell Spin Factor                0
Bubble Size                     0
Iron Binding Agent (kg)         0
Binder Activator (kg)           0
Flotation Column 01 Air Flow    0
Flotation Column 02 Air Flow    0
Flotation Column 03 Air Flow    0
Flotation Column 04 Air Flow    0
Flotation Column 05 Air Flow    0
Flotation Column 06 Air Flow    0
Flotation Column 07 Air Flow    0
Flotation Column 01 Level       0
Flotation Column 02 Level       0
Flotation Column 03 Level       0
Flotation Column 04 Level       0
Flotation Column 05 Level       0
Flotation Column 06 Level       0
Flotation Column 07 Level       0
% Iron Concentrate              0
% Silica Concentrate            0
% Iron Concentrate 2            0
% Silica Conce

/tmp/ipykernel_2665/499030973.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[columns_to_impute] = imputer.fit_transform(df_filtered[columns_to_impute])


In [15]:
df_filtered.isna().sum()

ts                              0
% Iron Feed                     0
% Silica Feed                   0
Starch Flow                     0
Amina Flow                      0
Ore Pulp Flow                   0
Ore Pulp pH                     0
Ore Pulp Density                0
Cell Spin Factor                0
Bubble Size                     0
Iron Binding Agent (kg)         0
Binder Activator (kg)           0
Flotation Column 01 Air Flow    0
Flotation Column 02 Air Flow    0
Flotation Column 03 Air Flow    0
Flotation Column 04 Air Flow    0
Flotation Column 05 Air Flow    0
Flotation Column 06 Air Flow    0
Flotation Column 07 Air Flow    0
Flotation Column 01 Level       0
Flotation Column 02 Level       0
Flotation Column 03 Level       0
Flotation Column 04 Level       0
Flotation Column 05 Level       0
Flotation Column 06 Level       0
Flotation Column 07 Level       0
% Iron Concentrate              0
% Silica Concentrate            0
% Iron Concentrate 2            0
% Silica Conce

#### Feature Engineering

In [16]:
# df_filtered[(df_filtered.ts> '2024-03-15 02:00:00') & (df_filtered.ts< '2024-03-15 03:00:00')]

In [17]:
# df_filtered[(df_filtered.ts> '2024-03-15 02:55:40') ].head(20)

In [18]:
# df_filtered[df_filtered['% Silica Concentrate 2']==1.36].head(60)

In [ ]:
# df_filtered[['ts', '% Silica Concentrate 2']].head(15)

,ts,% Silica Concentrate 2
8813,2024-03-15 02:00:00,1.36
8814,2024-03-15 02:00:20,1.36
8815,2024-03-15 02:00:40,1.36
8816,2024-03-15 02:01:00,1.36
8817,2024-03-15 02:01:20,1.36
8818,2024-03-15 02:01:40,1.36
8819,2024-03-15 02:02:00,1.36
8820,2024-03-15 02:02:40,1.36
8821,2024-03-15 02:03:00,1.36
8822,2024-03-15 02:03:20,1.36


In [ ]:
# df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 708640 entries, 8813 to 717452
Data columns (total 30 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   ts                            708640 non-null  datetime64[ns]
 1   % Iron Feed                   708640 non-null  float64       
 2   % Silica Feed                 708640 non-null  float64       
 3   Starch Flow                   708640 non-null  float64       
 4   Amina Flow                    708640 non-null  float64       
 5   Ore Pulp Flow                 708640 non-null  float64       
 6   Ore Pulp pH                   708640 non-null  float64       
 7   Ore Pulp Density              708640 non-null  float64       
 8   Cell Spin Factor              708640 non-null  float64       
 9   Bubble Size                   708640 non-null  float64       
 10  Iron Binding Agent (kg)       708640 non-null  float64       
 11  Binder Activato

In [ ]:
# df_filtered.shape

(708640, 30)

#### Training

In [19]:
def group_by_period(df, time_column, freq):

    aggregation = {}
    for column in df.columns:
        if column == time_column:
            continue
        if df[column].dtype == "object":
            print()
            aggregation[column] = lambda x: (
                x.mode().iloc[0] if not x.mode().empty else None
            )
        else:  # Numeric columns
            aggregation[column] = "mean"

    # Group by 15-minute intervals using pd.Grouper without setting index
    df_grouped = (
        df.groupby(pd.Grouper(key="ts", freq=freq))
        .agg(aggregation)
        .dropna(how="all")
        .reset_index()
    )
    return df_grouped

In [20]:
df_grouped = group_by_period(df_filtered, 'ts', "3600S")

/tmp/ipykernel_2665/1673687939.py:17: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  df.groupby(pd.Grouper(key="ts", freq=freq))


In [21]:
train_size = 0.8
train_index = int(len(df_grouped) * train_size)
train = df_grouped.iloc[:train_index]
test = df_grouped.iloc[train_index:]
print("Taille du train:", len(train))
print("Taille du test:", len(test))

target_iron = '% Iron Concentrate 2'
target_silicate = '% Silica Concentrate 2'

columns_to_drop = ['ts', '% Iron Concentrate', '% Silica Concentrate', '% Iron Concentrate 2',
       '% Silica Concentrate 2'] 
predictors = df_grouped.drop(columns=columns_to_drop).columns.tolist()

X_train = train[predictors]
X_test = test[predictors]
y_train_iron = train[target_iron]
y_train_silicate = train[target_silicate]
y_test_iron = test[target_iron]
y_test_silicate = test[target_silicate]

Taille du train: 3188
Taille du test: 797


In [22]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.linear_model import LinearRegression
import pickle

In [23]:
rf_iron = RandomForestRegressor(random_state=42)
rf_iron.fit(X_train, y_train_iron)
y_pred_iron = rf_iron.predict(X_test)
print("MAE : ", mean_absolute_error(y_test_iron, y_pred_iron))

MAE :  1.3927745294855696


In [24]:
with open('models/rf_iron.pkl', 'wb') as file:
    pickle.dump(rf_iron, file)

In [25]:
rf_silicate = RandomForestRegressor(random_state=42)
rf_silicate.fit(X_train, y_train_silicate)
y_pred_siliacte = rf_silicate.predict(X_test)
print("MAE : ", mean_absolute_error(y_test_silicate, y_pred_siliacte))

with open('models/rf_silicate.pkl', 'wb') as file:
    pickle.dump(rf_silicate, file)

MAE :  0.8622143036386449


In [38]:
X_test.columns

Index(['% Iron Feed', '% Silica Feed', 'Starch Flow', 'Amina Flow',
       'Ore Pulp Flow', 'Ore Pulp pH', 'Ore Pulp Density', 'Cell Spin Factor',
       'Bubble Size', 'Iron Binding Agent (kg)', 'Binder Activator (kg)',
       'Flotation Column 01 Air Flow', 'Flotation Column 02 Air Flow',
       'Flotation Column 03 Air Flow', 'Flotation Column 04 Air Flow',
       'Flotation Column 05 Air Flow', 'Flotation Column 06 Air Flow',
       'Flotation Column 07 Air Flow', 'Flotation Column 01 Level',
       'Flotation Column 02 Level', 'Flotation Column 03 Level',
       'Flotation Column 04 Level', 'Flotation Column 05 Level',
       'Flotation Column 06 Level', 'Flotation Column 07 Level'],
      dtype='object')

In [26]:
models = {
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
    'xgboost regressor': XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8, random_state=42)}

for name, model in models.items():
    model.fit(X_train, y_train_iron)
    y_pred_iron = model.predict(X_test)
    print(f"MAE {name}: ", mean_absolute_error(y_test_iron, y_pred_iron))
    print(f"MAPE {name}: ", mean_absolute_percentage_error(y_test_iron, y_pred_iron))

MAE Random Forest:  1.3927745294855696
MAPE Random Forest:  0.022720119220080586
MAE Gradient Boosting:  1.4396611074091175
MAPE Gradient Boosting:  0.02349065127891514
MAE xgboost regressor:  1.514025905326738
MAPE xgboost regressor:  0.024743379930689095


In [27]:
for name, model in models.items():
    model.fit(X_train, y_train_silicate)
    y_pred_silicate = model.predict(X_test)
    print(f"MAE {name}: ", mean_absolute_error(y_test_silicate, y_pred_silicate))
    print(f"MAPE {name}: ", mean_absolute_percentage_error(y_test_silicate, y_pred_silicate))

MAE Random Forest:  0.8622143036386449
MAPE Random Forest:  0.36833198335240375
MAE Gradient Boosting:  0.846279331839276
MAPE Gradient Boosting:  0.3420448440344709
MAE xgboost regressor:  0.8413681965281105
MAPE xgboost regressor:  0.36614395130035554


In [34]:
# import plotly.graph_objects as go

# def plot(test_timestamp, y_true, y_pred):
#     fig = go.Figure()
#     fig.add_trace(
#         go.Scatter(
#             x=test_timestamp,
#             y=y_true,
#             mode="lines",
#             name="Errors Recup",
#             line=dict(color="blue"),
#         )
#     )
#     fig.add_trace(
#         go.Scatter(
#             x=test_timestamp,
#             y=y_pred,
#             mode="lines",
#             name="Errors Silice",
#             line=dict(color="red"),
#         )
#     )

#     fig.update_layout(
#         title="Errors Recup and Silice",
#         xaxis_title="Time",
#         yaxis_title="Error value",
#         xaxis_tickangle=-45,
#         legend_title="Errors",
#     )

#     fig.show()

In [35]:
# plot(test.ts, y_test_iron, y_pred_iron)

In [33]:
# plot(test.ts, y_test_silicate, y_pred_silicate)

In [36]:
# def calculate_new_trend_score(
#     y_test: pd.Series, y_pred: pd.Series, thrsh: float
# ) -> float:

#     assert len(y_test) == len(y_pred)
#     # Create temporary dataframe
#     df = pd.DataFrame([y_test.values, y_pred]).T
#     df.columns = ["y_true", "y_pred"]
#     df["diff_from_previous_value_y_true"] = df["y_true"] - df["y_true"].shift(
#         1
#     )
#     df["diff_from_previous_value_y_pred"] = df["y_pred"] - df["y_pred"].shift(
#         1
#     )
#     df["drop_y_true"] = df["diff_from_previous_value_y_true"] < 0
#     df["drop_y_pred"] = df["diff_from_previous_value_y_pred"] < 0
#     df["same_direction"] = (df["drop_y_true"] == df["drop_y_pred"]) | (
#         (abs(df["diff_from_previous_value_y_true"]) < thrsh)
#         & (abs(df["diff_from_previous_value_y_pred"]) < thrsh)
#     )

#     trend_score = len(df.loc[df.same_direction == True]) / len(df)

#     return trend_score